# FAST-UAV - Off-design mission performance
*Author: Félix Pollet - 2023* <br>

This notebook provides an example of how to compute the performance of a pre-defined UAV design for any operational mission.

For this purpose, we will re-use the results from a previous design optimization process, stored in the [problem_outputs_DJI_M600_mdo.xml](./data/source_files/problem_outputs_DJI_M600_mdo.xml) file. The present problem is slightly different from that used for design optimization. In fact, it consists of a simple evaluation of the performance module. No optimization of any kind is performed, since the design is already known/fixed.

In [ ]:
# Import required librairies
import os.path as pth
import sys
import logging
import fastoad.api as oad
import shutil
from fastuav.utils.postprocessing.analysis_and_plots import energy_breakdown_sun_plot_drone

sys.path.append(pth.abspath("."))

#logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

# For using all screen width
from IPython.display import display, HTML, IFrame
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
# Paths to folders and files
DATA_FOLDER_PATH = "data"
WORK_FOLDER_PATH = "workdir"
CONFIGURATION_FOLDER_PATH = pth.join(DATA_FOLDER_PATH, "configurations")
SOURCE_FOLDER_PATH = pth.join(DATA_FOLDER_PATH, "source_files")

CONFIGURATION_FILE = pth.join(CONFIGURATION_FOLDER_PATH, "multirotor_performance.yaml")   # The problem is now a simple evaluation of the UAV performance, not an optimization
SOURCE_FILE = pth.join(SOURCE_FOLDER_PATH, "problem_outputs_DJI_M600_mdo.xml")  # The source file is the output file of a previous design optimization process.

## 1. Setting up the performance analysis

Have a look at the [multirotor_performance.yaml](./data/configurations/multirotor_performance.yaml) file. The model used for the performance analysis solely consists of the `Missions` module. No design models for the components is used, only the performance models.<br>

```yaml
model:
    missions:
        id: fastuav.performance.mission
        file_path: ../missions/missions_multirotor.yaml
```

The definition of the missions, routes and flight phases are defined in the [mission file](./data/missions/missions_multirotor.yaml) whose path is provided in the configuration file. A mission is defined as an assembly of routes (e.g. a main route and a diversion flight), which are in turn defined as an assembly of flight phases. Each flight phase is identified by a `phase_id` that describes it. The phase identifiers associated with the multirotor drones are the `vertical climb`, the `multirotor cruise` and the `hover`.

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, "n2.html")
oad.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width="100%", height="500px")

## 2. Run the performance analysis

To run a performance analysis, you need an XML file describing the drone design. As explained earlier, this can be the output of a design problem (see the [Multirotor Design Notebook](./1_Multirotor_Design.ipynb)).

Let's generate the input file for the problem:

In [ ]:
input_file = oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

The warning message indicates that some variables related to the mission profile have not yet been assigned a value. Using the variable viewer, provide the values for the missions you want to evaluate. For example, the first operational mission may consist of 10,000 meters cruise distance with a payload mass of 3 kg (instead of the 5.5 design payload mass), at an altitude of 150 meters and a flight speed of 15 m/s.

In [ ]:
oad.variable_viewer(input_file)

You can use the `VariableViewer` to fill in the values for the mission profile, in the 'mission' menu (do not forget to save!).

Once, it is done, we can run the problem using `evaluate_problem`...

In [ ]:
eval_problem = oad.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

... and have a look at the results.

In [ ]:
output_file = eval_problem.output_file_path
oad.variable_viewer(output_file)

## 3. Analysis and plots

For the moment, the post processing plots are limited to the visualization of the energy consumed over the mission. If several mission have been computed, you can compare them on the same plot.

In [ ]:
mission_name = "operational_1"  # name of the mission to plot
fig = energy_breakdown_sun_plot_drone(output_file, mission_name=mission_name)
fig.show()

In [ ]:
mission_name = "operational_2"  # secondary mission
fig = energy_breakdown_sun_plot_drone(output_file, mission_name=mission_name, fig=fig)
fig.show()